<a href="https://colab.research.google.com/github/Snoke9/MLaDA/blob/main/binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 3
# Бинарная классификация
**Цель работы:** познакомиться с применением модели машинного обучения для решения задачи классификации (метод логистической регрессии)

Выполним анализ набора данных «Выживаемость пациентов». Набор данных содержит информацию о выживаемости пациенток, перенесших операцию по поводу рака молочной железы. Набор содержит случаи из исследования, проводившегося с 1958 по 1970 год в больнице Биллингса Чикагского университета.


Набор данных включает следующие атрибуты:

* **age** – возраст пациента на момент операции (целое число)
* **year** – год операции пациента (целое число)
* **nodes** – количество обнаруженных положительных подмышечных узлов (целое число)
* **survival** – статус выживания (целевая переменная), где 0 – означает, что пациент умер в течение 5 лет, а 1 – означает, что пациент прожил 5 лет или дольше.



### Импортируем библиотеки и загружаем данные

In [ ]:
import numpy as mp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/haberman.csv')
# заменим двойки на нули
df['survival'] = df['survival'].map({1: 1, 2: 0})
df.head()

Выведем целевую переменную, целевые имена и частоту каждого уникального значения

In [ ]:
(unique, counts) = np.unique(df['survival'], return_counts=True)
print(f'Уникальные значения целевой переменной: {unique}')
print(f'Количество записей целевой переменной: {counts}')

Построим столбчатую диаграмму, чтобы увидеть целевую переменную:

In [ ]:
sns.set_style('whitegrid')
sns.countplot(df, x='survival')

Анализ диаграммы показывает, что набор данных является сбалансированным

In [ ]:
df.info()

В этом наборе данных у нас есть два класса: 0 и 1, что делает эту задачу задачей бинарной классификации

Выполним бинарную классификацию с использованием логистический регрессии с помощью sklearn



#### Проверяем наличие пропущенных значений и выбросов

In [ ]:
#проверяем на пропущенные значения
print(df.isnull().sum())

In [ ]:
#проверяем на выбросы
sns.boxplot(x=df['age'])

#### Разделяем данные на признаки и целевую переменную

In [ ]:
df_copy = df
y = df_copy['survival']
X = df_copy.drop('survival', axis=1)

In [ ]:
X

#### Применяем операции нормализации для численной устойчивости

In [ ]:
from sklearn.preprocessing import StandardScaler
standartizer = StandardScaler()
X = standartizer.fit_transform(X)

In [ ]:
X

### Разделяем данные на тренировочный и тестовый наборы

82% данных используется для обучения и 18% — для тестирования.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=0)

In [ ]:
# создаем экземпляр модели с учетом взвешенности классов
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

#### Рассчитываем показатели точности, сравнив фактические и прогнозируемые значения

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()

print(f'True Positive(TP) = {TP}')
print(f'False Positive(FP) = {FP}')
print(f'True Negative(TN) = {TN}')
print(f'False Negative(FN) = {FN}')

accuracy = (TP + TN) / (TP + FP + TN + FN)
print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

In [ ]:
# оцениваем точность модели
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)

In [ ]:
sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f'Точность: {precision:.2f}')
print(f'Полнота: {recall:.2f}')

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
print(f'F1-мера: {f1:.2f}')

In [ ]:
df_copy.corr()

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# Получаем предсказание вероятности для класса 1
y_score = model.predict_proba(X_test)[:, 1]

In [ ]:
# Рассчитываем FPR, TPR и пороги
fpr, tpr, thresholds = roc_curve(y_test, y_score)

In [ ]:
# Рассчитываем AUC - площадь под кривой
roc_auc = auc(fpr, tpr)

In [ ]:
# Строим график
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Reciever Oparating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()